In [1]:
import data_generator as dgtr
import torchvision.models as models
import numpy as np
import torch
import torch.nn as nn

In [2]:
train_d, val_d, train_data, test_data, classList = dgtr.generate_data()

Read 207 images
Read 208 images


In [3]:
resnet34 = models.resnet34(pretrained=True)

In [4]:
class fcn(nn.Module):
    def __init__(self, num_classes=len(classList)):
        super(fcn, self).__init()
        
        self.layer1 = nn.Sequential(*list(resnet34.children())[:-4])
        self.layer2 = list(resnet34.children())[-4]
        self.layer3 = list(resnet34.children())[-3]
        
        self.conv1 = nn.Conv2d(512, num_classes, 1)
        self.conv2 = nn.Conv2d(256, num_classes, 1)
        self.conv3 = nn.Conv2d(128, num_classes, 1)
        
        self.upsample_8x = nn.ConvTranspose2d(num_classes, num_classes, 16, 8, 4, bias=False)
        self.unsample_8x.weight.data = bilinear_kernal(num_classes, num_classes, 16)
        
        self.upsample_4x = nn.ConvTranspose2d(num_classes, num_classes, 4, 2, 1, bias=False)
        self.unsample_4x.weight.data = bilinear_kernal(num_classes, num_classes, 4)
        
        self.upsample_2x = nn.ConvTranspose2d(num_classes, num_classes, 4, 2, 1, bias=False)
        self.unsample_2x.weight.data = bilinear_kernal(num_classes, num_classes, 4)
        
    def bilinear_kernal(self, in_channels, out_channels, kernel_size):
        factor = (kernel_size + 1) // 2
        if kerne_size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        # return open multi-dimensional meshgrid
        og = np.ogrid[:kernel_size, :kernel_size]
        filt = (1 - abs(og[0] - center) / factor) * (
            1 - abs(og[1] - center) / factor)
        weight = np.zeros((in_channels, out_channels, kernel_size),
                          dtype='float32')
        weight[range(in_channels), range(out_channels), :, :] = filt
        return torch.from_numpy(weight)
    
    def forward(self, x):
        # 1/8
        x = self.layer1(x)
        cv1 = x
        
        # 1/16
        x = self.layer2(x)
        cv2 = x
        
        # 1/32
        x = self.layer3(x)
        cv3 = x
        
        cv3 = self.conv1(cv3)
        cv3 = self.unsample2x(cv3)
        cv2 = self.conv2(cv2)    
        cv2 = cv2 + cv3
        cv1 = self.conv3(cv1)
        cv2 = self.unsample4x(cv2)
        out = cv2 + cv1
        out = self.unsample_8x(out)
        return out
        

SyntaxError: can't use starred expression here (<ipython-input-9-a8b5f8312d5a>, line 4)

In [19]:
nn.ConvTranspose2d?

Init signature:
nn.ConvTranspose2d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    output_padding=0,
    groups=1,
    bias=True,
    dilation=1,
    padding_mode='zeros',
)
Docstring:     
Applies a 2D transposed convolution operator over an input image
composed of several input planes.

This module can be seen as the gradient of Conv2d with respect to its input.
It is also known as a fractionally-strided convolution or
a deconvolution (although it is not an actual deconvolution operation).

* :attr:`stride` controls the stride for the cross-correlation.

* :attr:`padding` controls the amount of implicit zero-paddings on both
  sides for ``dilation * (kernel_size - 1) - padding`` number of points. See note
  below for details.

* :attr:`output_padding` controls the additional size added to one side
  of the output shape. See note below for details.

* :attr:`dilation` controls the spacing between the kernel points; also known as the à trous algori